# XGBoost
XGBoost 是 eXtreme Gradient Boosting 的缩写称呼，它是一个非常强大的 Boosting 算法工具包，优秀的性能（效果与速度）让其在很长一段时间内霸屏数据科学比赛解决方案榜首，现在很多大厂的机器学习方案依旧会首选这个模型。

XGBoost 在并行计算效率、缺失值处理、控制过拟合、预测泛化能力上都变现非常优秀

## 模型原理

XGBoost也是一个加法模型，但是它结点分裂的标准发生了新的变化。无论是回归问题还是分类问题，我们都先定义一个损失函数 
 $L(y, \hat y)$，损失函数是可微的任意损失函数。

目标函数（Objective function） $Obj(\Theta) = L(\Theta) + \Omega(\Theta)$
 + $L(\Theta) $代表训练损失函数（Training Loss），表示模型多好的拟合了训练数据。
 + $\Omega(\Theta)$为正则化项（Regularization）衡量了模型的复杂程度。

$\displaystyle obj = \sum_{i=1}^n L(y_i,\hat y_i)  + \sum_{k=1}^K\Omega (f_k)$
+ $L(\Theta) = \sum_{i=1}^n L(y_i,\hat y_i)$
+ $\Omega(\Theta) = \sum_{k=1}^K\Omega (f_k)$
+ $\hat y_i = \sum_{k=1}^K f_k(x_i)$
+ n个样本， K棵树

### 解决方案
$
\begin{align} \\
\displaystyle
\hat y_i &= \sum_{k=1}^K f_k(x_i) \\
\hat y_i^{(0)} &= 0 \\
\hat y_i^{(1)} &= f_1(x_i) = \hat y_i^{(0)} + f_1(x_i) \\
\hat y_i^{(2)} &= f_1(x_i) + f_2(x_i) = \hat y_i^{(1)} + f_2(x_i) \\
\hat y_i^{(t)} &= \sum_{k=1}^t f_k(x_i)= \hat y_i^{(t-1)} + f_t(x_i) \\
\end{align}
$

F为包含所有回归树的函数空间\
$\hat y_i^{(t)}$ 是第 t 次迭代的预测值。 \
$f_t(x_i)$是第  t 颗树，也就是我们第  t 次迭代需要得到的树。

### 目标函数变换

$
\begin{align}
\displaystyle
obj^{(t)} &= \sum_{i=1}^n L(y_i,\hat y_i^{(t)})  + \sum_{k=1}^t\Omega (f_k)\\
          &= \sum_{i=1}^n L(y_i, \hat y_i^{(t-1)} + f_t(x_i))  + \Omega (f_k) + constant， f_1 到f_{t-1} 作为常数， 假设L是平方损失\\
          &= \sum_{i=1}^n (y_i - \hat y_i^{(t-1)} - f_t(x_i))^2 + \Omega (f_k) + constant \\
          &= \sum_{i=1}^n (2(y_i - \hat y_i^{(t-1)})f_t(x_i) + f_t(x_i)^2) + \Omega (f_k) + constant, (y_i - \hat y_i^{(t-1)})^2 作为常数\\
\end{align}
$

泰勒公式二阶展开 \
$f(x+\Delta x) \approx f(x) + f^{'}(x)\Delta x + \frac{1}{2} f^{''}(x) \Delta x^2$ 

$假设 f(\hat y_i^{t}) = L(y_i,\hat y_i^{(t)}) = (y_i - \hat y_i^{(t)}) ^2, \Delta x = f_t(x_i) $ \
$f(\hat y_i^{t}) \approx f(\hat y_i^{(t-1)} + f_t(x_i)) = f(\hat y_i^{(t-1)}) + f^{'}(\hat y_i^{(t-1)}) f_t(x_i) + \frac{1}{2} f^{''}(\hat y_i^{(t-1)})  f_t(x_i)^2  $\
$g_i = f^{'}(\hat y_i^{(t-1)}) = \frac {\partial f(\hat y_i^{(t-1)})}{\partial \hat y_i^{(t-1)}}$\
$h_i =f^{''}(\hat y_i^{(t-1)}) = \frac {\partial ^2 f(\hat y_i^{(t-1)})}{\partial \hat y_i^{(t-1)}}  $


$
\begin{align}
\displaystyle
obj^{(t)} &\approx  \sum_{i=1}^n  (L(y_i,\hat y_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2) + \Omega (f_k) + constant \\
          &= \sum_{i=1}^n (g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2) + \Omega (f_k) + constant, L(y_i,\hat y_i^{(t-1)})作为常量
\end{align}
$

目标函数：linear\
目标函数：$L(y_i, \hat y_i^{(t-1)}) = \frac {1}{2} *（\hat y_i^{(t-1)}  - y） ^ 2$ \
一阶导数（grad）：$ g_i = \hat y_i - y_i$\
二阶导数（hess）： $h_i = 1$

目标函数：logistic\
目标函数：$L(y_i, \hat y_i^{(t-1)}) = y_i * ln(1 + exp(-\hat y_i^{(t-1)})) + (1 - y_i) * ln(1+exp(\hat y_i^{(t-1)}))$\
一阶导数（grad）：$g_i = sigmoid(\hat y_i^{(t-1)}) - y_i$\
二阶导数（hess）：$h_i = sigmoid(\hat y_i^{(t-1)}) * (1 - sigmoid(\hat y_i^{(t-1)}))$

### 重新定义树
我们重新定义一下树：我们通过叶子结点中的分数向量和将实例映射到叶子结点的索引映射函数来定义树：\
$f_t(x) = w_q(x), w\in R^T, q:R^d -> {1,2,3....T}$
+ w代表树中叶子结点的权重
+ q代表的是树的结构

<img src="../data/img/xgb_wq.png" style="zoom:40%"/>

### 定义树的复杂程度（不唯一）
$\Omega f_t = \gamma T + \frac{1}{2} \lambda \sum_{j=1} ^T w_j^2 $
+ $\gamma T$ 代表了叶子结点的个数
+ 叶子结点分数的 L2 Norm, $w_j$ 是叶子节点j的值和权重乘积

### 重新审视目标函数
定义在叶子结点 j 中的实例(样本）的集合为: $I_j=\{i|q(x_i)=j\}$

$\displaystyle \sum_{i=1}^n g_i w_q(x_i) => \sum_{j=1}^T (\sum_{i\in I_j} g_i) w_j$

$
\begin{align}
\displaystyle
obj^{(t)} &\approx \sum_{i=1}^n (g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2) + \Omega (f_k) + constant\\
          &= \sum_{i=1}^n (g_i w_q(x_i) + \frac{1}{2} h_i w_q(x_i)^2) + \gamma T + \frac{1}{2} \lambda \sum_{j=1} ^T w_j^2 + constant\\
          &=\sum_{j=1}^T [(\sum_{i\in I_j} g_i) w_j +  \frac{1}{2}(\sum_{i\in I_j} h_i) w_j^2 ] + \gamma T + \frac{1}{2} \lambda \sum_{j=1} ^T w_j^2 + constant\\
          &=\sum_{j=1}^T [(\sum_{i\in I_j} g_i) w_j +  \frac{1}{2}(\sum_{i\in I_j} h_i + \lambda) w_j^2 ] + \gamma T  + constant
\end{align}
$

上式是 T 个独立二次函数的和

### 计算叶子结点的值
$f(x) = Gx + \frac{1}{2} H x^2 => 求导 G + Hx = 0 => x = - \frac{G}{H} => f(x)最小值 -\frac{1}{2} \frac{G^2}{H}$

重新定义目标函数\
$G_j= \sum_{i\in I_j} g_i$\
$H_j = \sum_{i\in I_j} h_i $

$
\begin{align}
\displaystyle
obj^{(t)} &=\sum_{j=1}^T [(\sum_{i\in I_j} g_i) w_j +  \frac{1}{2}(\sum_{i\in I_j} h_i + \lambda) w_j^2 ] + \gamma T  + constant\\
          &=\sum_{j=1}^T [G_j w_j +  \frac{1}{2}(H_j + \lambda) w_j^2 ] + \gamma T  + constant
\end{align}
$

假设树的结构 q(x) 是固定的，那么每一个叶子结点的权重的最优值为\
$w_j^*=- \frac{G_j}{H_j + \lambda} $

目标函数的最优值为
$obj^{(t)} = -\frac{1}{2} \sum_{j=1}^T \frac{G_j^2}{H_j +\lambda} + \gamma T$


### 贪婪算法生成树

首先生成一个深度为 1 的树（只有一个根结点，也叫叶子结点）

XGBoost 子树的基本处理思路和 CART 一样，会对特征值排序后遍历划分点，将其中最优的分裂收益作为该特征的分裂收益，选取具有最优分裂收益的特征作为当前节点的划分特征，按其最优划分点进行二叉划分，得到左右子树。

对于每棵树的每个叶子结点，尝试去做分裂（生成两个新的叶子结点，原来的叶子结点不再是叶子结点）。在增加了分裂后的目标函数前后变化为（我们希望增加了树之后的目标函数小于之前的目标函数，所以用之前的目标函数减去之后的目标函数）：
 $$
 Gain = \frac{1}{2} (\frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L + G_R)^2}{H_L + H_R + \lambda} ) - \gamma
 $$
 + $\frac{G_L^2}{H_L + \lambda}$ 是左面叶子结点的值
 + $\frac{G_R^2}{H_R + \lambda}$ 是右面叶子结点的值
 + $\frac{(G_L + G_R)^2}{H_L + H_R + \lambda}$是未分裂前的值
 + $\gamma$是引入了多一个的叶子结点增加的复杂度

<img src="../data/img/xgb_tree.png" style="zoom:40%"/>

#### 近似算法
基于性能的考量，XGBoost 还对贪心准则做了一个近似版本，简单说，处理方式是「将特征分位数作为划分候选点」。这样将划分候选点集合由全样本间的遍历缩减到了几个分位数之间的遍历。

展开来看，特征分位数的选取还有 global 和 local 两种可选策略：

global 在全体样本上的特征值中选取，在根节点分裂之前进行一次即可；
local 则是在待分裂节点包含的样本特征值上选取，每个节点分裂前都要进行。
这两种情况里，由于global只能划分一次，其划分粒度需要更细。



XGB 原始 paper 中对 Higgs Boson 数据集进行了实验，比较了精确贪心准则、global 近似和 local 近似三类配置的测试集 AUC，用 eps 代表取分位点的粒度，如 eps=0.25 代表将数据集划分为1/0.25 = 4  个buckets，发现 global（ eps=0.05 ）和local（ eps=0.3 ）均能达到和精确贪心准则几乎相同的性能。



### 列采样与学习率
XGBoost为了进一步优化效果，在以下2个方面进行了进一步设计：

列采样
+ 和随机森林做法一致，每次节点分裂并不是用全部特征作为候选集，而是一个子集。
+ 这么做能更好地控制过拟合，还能减少计算开销。
学习率
+ 也叫步长、shrinkage，具体的操作是在每个子模型前（即每个叶节点的回归值上）乘上该系数，不让单颗树太激进地拟合，留有一定空间，使迭代更稳定。XGBoost默认设定为 。


### 特征缺失与稀疏性
XGBoost 也能对缺失值处理，也对特征稀疏问题（特征中出现大量的  或one-hot encoding结果）做了一些优化。XGBoost 用「稀疏感知」策略来同时处理这两个问题：

+ 简单说，它的做法是将缺失值和稀疏  值等同视作缺失值，将其「绑定」在一起，分裂节点的遍历会跳过缺失值的整体。这样大大提高了运算效率。
+ 值在XGB中被处理为数值意义上的  还是 ，需要结合具体平台的设置，预处理区分开作为数值的  （不应该被处理为  ）和作为稀疏值的  （应该被处理为  ）。
   
 https://www.showmeai.tech/article-detail/194

## XGBoost工程优化

1. 并行列块设计
XGBoost 将每一列特征提前进行排序，以块（Block）的形式储存在缓存中，并以索引将特征值和梯度统计量对应起来，每次节点分裂时会重复调用排好序的块。而且不同特征会分布在独立的块中，因此可以进行分布式或多线程的计算。

<img src="../data/img/xgb_cal.png" style="zoom:40%"/>

2. 缓存访问优化
特征值排序后通过索引来取梯度$g_i$, $h_i$ 会导致访问的内存空间不一致，进而降低缓存的命中率，影响算法效率。为解决这个问题，XGBoost为每个线程分配一个单独的连续缓存区，用来存放梯度信息。

3. 核外块计算
数据量非常大的情形下，无法同时全部载入内存。XGBoost 将数据分为多个 blocks 储存在硬盘中，使用一个独立的线程专门从磁盘中读取数据到内存中，实现计算和读取数据的同时进行。
为了进一步提高磁盘读取数据性能，XGBoost 还使用了两种方法：

① 压缩 block，用解压缩的开销换取磁盘读取的开销。\
② 将 block 分散储存在多个磁盘中，提高磁盘吞吐量。

### XGBoost vs GBDT
<img src="../data/img/xgb_vs_gbdt.png" style="zoom:40%"/>

## XGBoost 特征处理
论是XGBoost还是其他的Boosting Tree，使用的Tree都是cart回归树，
这也就意味着该类提升树算法只接受连续型特征和离散特征中的数值型特征，不直接支持离散特征中的类别型特征。
默认情况下，xgboost会把类别型的特征当成数值型。

总结起来的结论，大至两条：
- 对于有序的类别型特征，比如 age 等，当成数值型变量处理可以的，用label encode
  对于无序的类别型特征，推荐one-hot。但是one-hot会增加内存开销以及训练时间开销(因为1列变成多列)
- 类别型变量在范围较小时（tqchen 给出的是[10,100]范围内）推荐使用。

xgboost是不支持category特征的，在训练模型之前，需要我们进行预处理，可以根据特征的具体形式来选择：
- 有序特征：label encoding，比如版本号
- 无序特征：one-hot encoding，比如城市
